In [1]:
pdf_path = '../resumes/Riyanshi%20Bohra%20-%20Resume.pdf'

In [ ]:
import pymupdf4llm
import pymupdf
from IPython.display import Markdown, display

file = pymupdf.open(pdf_path)

md_text = pymupdf4llm.to_markdown(
    file  
)

display(Markdown(md_text))

In [ ]:
from doctr.models import ocr_predictor, from_hub
from doctr.io import DocumentFile

docs = DocumentFile.from_pdf(pdf_path)
model = from_hub('Felix92/doctr-torch-parseq-multilingual-v1')
predictor =  ocr_predictor(
    det_arch='fast_base',
    reco_arch=model,
    pretrained=True,
    assume_straight_pages=True,
    detect_orientation=False,
    
)

result = predictor(docs)
result.show()

In [ ]:
text = result.render()

print(text)

In [ ]:
from pypdf import PdfReader

with open(pdf_path, 'rb') as f:
    reader = PdfReader(f)
    content = "\n".join([x.extract_text() for x in reader.pages])

content

In [2]:
from file_loader import FileLoader

loader = FileLoader()
propositions = loader.load_pdf(pdf_path)
propositions

/home/joseph/joyfuljobs/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 85163.53it/s]
/home/joseph/joyfuljobs/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


["Riyanshi Bohra's email is riyanshibohraa@gmail.com.",
 "Riyanshi Bohra's phone number is 352-757-8079.",
 "Riyanshi Bohra's LinkedIn profile can be found at www.linkedin.com/in/riyanshi-bohra.",
 'Riyanshi Bohra is pursuing a Master of Science (M.S.) in Data Science.',
 "Riyanshi Bohra's M.S. program is from August 2023 to May 2025.",
 'Riyanshi Bohra is studying at the University of Arizona, School of Information.',
 "Riyanshi Bohra's GPA for the M.S. program is 3.7 out of 4.0.",
 'The University of Arizona is located in Tucson, Arizona.',
 "Riyanshi Bohra's relevant coursework includes Data Mining, Data Visualization, Machine Learning, Natural Language Processing (NLP), and Artificial Intelligence.",
 'Riyanshi Bohra completed a Senior Certificate Program in Computer Science & Engineering.',
 "Riyanshi Bohra's Senior Certificate Program was from January 2023 to May 2023.",
 'Riyanshi Bohra attended the University of Florida, Herbert Wertheim College of Engineering for the Senior Ce

In [5]:
from gliner import GLiNER
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from prompts import STRAIGHT_FORWARD_PROMPT
import re

ner = GLiNER.from_pretrained("urchade/gliner_large-v2.1")
labels = ["name", "phone number", "university/college", "college_degree"]
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
chain = ChatPromptTemplate.from_messages([("system", STRAIGHT_FORWARD_PROMPT ), ("human", "{proposition}\n\nWhat is the full name of the college degree only? (ex: Bachelor of Science in Data Science, Master of Science in Aerospace Engineering, B.A. in Psychology, Master's in English, Master of Fine Arts, Bachelor of Arts in Music)")]) | llm 

def extract_person_metadata(propositions: list[str]) -> str:
    data: dict[str, list[set[str]]] = {
        "email": [set(), set()],
    }
    for proposition in propositions:
        found_email = re.search("(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|\"(?:[\\x01-\\x08\\x0b\\x0c\\x0e-\\x1f\\x21\\x23-\\x5b\\x5d-\\x7f]|\\\\[\\x01-\\x09\\x0b\\x0c\\x0e-\\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\\x01-\\x08\\x0b\\x0c\\x0e-\\x1f\\x21-\\x5a\\x53-\\x7f]|\\\\[\\x01-\\x09\\x0b\\x0c\\x0e-\\x7f])+)\\])", proposition.lower())
        if found_email is not None:
            data["email"][1].add(found_email.group(0))
        entities = ner.predict_entities(proposition, labels, threshold=0.5)
        for entity in entities:
            key, value = entity["label"], entity["text"]
            if key not in data:
                data[key] = [set(), set()]

            match key:
                case "university/college":
                    value = value.lower()
                    if "college" in value or "university" in value:
                        data[key][1].add(value.replace("the", "").strip().title())
                case "college_degree":
                    value = value.lower()
                    if value not in data[key][1]:
                        data[key][0].add(proposition)
                        answer = chain.invoke({ "proposition": proposition }).content
                        if answer != "None":
                            print(f"\"{proposition}\" => \"{answer}\"")
                            data[key][1].add(answer)
                case _:
                    data[key][1].add(value)
    for key in data:
        data[key] = ", ".join([x.rstrip('.') for x in data[key][1]])
    return data

extract_person_metadata(propositions)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 41734.37it/s]
/home/joseph/joyfuljobs/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


"Riyanshi Bohra is pursuing a Master of Science (M.S.) in Data Science." => "Master of Science (M.S.)"
"Riyanshi Bohra is pursuing a Master of Science (M.S.) in Data Science." => "Master of Science (M.S.) in Data Science"
"Riyanshi Bohra's M.S. program is from August 2023 to May 2025." => "M.S."
"Riyanshi Bohra's GPA for the M.S. program is 3.7 out of 4.0." => "M.S."
"Riyanshi Bohra completed a Senior Certificate Program in Computer Science & Engineering." => "Senior Certificate Program"
"Riyanshi Bohra's Senior Certificate Program was from January 2023 to May 2023." => "Senior Certificate Program"
"Riyanshi Bohra attended the University of Florida, Herbert Wertheim College of Engineering for the Senior Certificate Program." => "Senior Certificate Program"
"Riyanshi Bohra's GPA for the Senior Certificate Program is 4.0 out of 4.0." => "Senior Certificate Program"
"Riyanshi Bohra holds a Bachelor of Technology (B. Tech) in Information Technology with a Minor in Data Science." => "Bachel

{'email': 'riyanshibohraa@gmail.com',
 'name': 'Riyanshi Bohra',
 'phone number': '352-757-8079',
 'college_degree': 'Master of Science (M.S.) in Data Science, Master of Science (M.S.), B. Tech, Senior Certificate Program, M.S, Bachelor of Technology',
 'university/college': 'University Of Florida, Mel And Enid Zuckerman College Of Public Health, Herbert Werim College Of Engineering, Manipal University Jaipur, University Of Arizona'}

In [ ]:
# from gliner import GLiNER

# ner = GLiNER.from_pretrained("urchade/gliner_large-v2.1")

# labels = ["name"]

# entities = ner.predict_entities(, labels, threshold=0.1)

# for entity in entities:
#     print(entity["label"], "=>", entity["text"])
    